<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Structure-cheat-sheet" data-toc-modified-id="Structure-cheat-sheet-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Structure cheat sheet</a></span></li><li><span><a href="#Data-structure" data-toc-modified-id="Data-structure-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data structure</a></span></li><li><span><a href="#get-features" data-toc-modified-id="get-features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>get features</a></span></li></ul></div>

# Basics

## Structure cheat sheet

1. func: train data lead (following order)
    1. read the descriptive dataframe from the feature-pipeline
    2. extract feature from the feature-objects which are labeled train-dataset from dataframe
    3. create numpy feature array for the processing pipeline
2. preprocessing
    1. Transformation (any combination of the following)
        + log-transform
        + PCA
        + others
    2. Scaling (one of the following)
        + StandardScaler
        + MinMaxScaler
3. Unsupervised Clustering
    1. Estimate initial hyperparameter
    2. Create grid over various hyperparameters
    3. Train all and choose the best according to metric
    
    
in all steps the cluster-recorder object (possibly dataframe-row) will record all the meta-information like hyper-parameters

## Data structure

There are multiple degrees of freedom in the data:

1. Signal to noise ratio (SNR)
2. Machine type
    1. pump
    2. fan
    3. valve (solenoid)
    4. slider
3. Machine ID
    1. four different machine IDs
    
The pipeline will be applied to fixed SNR, fixed machine type and fixed ID

## get features

Get the descriptive dataframe for the features.

The descriptive dataframe contains all IDs of the pump. We will focus on ID '00' for now since the modeling phase is seperated per SNR, per machine, per ID anyway.

class: 
+ uni\_\<model\>
attributes:
+ default threshold
+ roc_auc
methods:
+ fit
+ predict
+ predict_score
+ eval_roc_auc

In [1]:
#===============================================
# Basic Imports
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import sys
import glob
from tqdm.auto import tqdm 

import datetime
import time
import logging

sns.set()

BASE_FOLDER = '../../'
%run -i ..\..\utility\feature_extractor\JupyterLoad_feature_extractor.py
%run -i ..\..\utility\modeling\JupyterLoad_modeling.py

load feature_extractor_mother
load feature_extractor_mel_spectra
load feature_extractor_psd
load feature_extractore_pre_nnFilterDenoise
load extractor_diagram_mother


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FastICA

preprocessing = [
    (FastICA, {'n_components':40, 'algorithm':'parallel'}),
    (StandardScaler, {})
]

modeling = (uni_EllipticEnvelope, {'random_state':42})

IDs = [
    '00',
    '02',
    '04',
    '06'
      ]

machines = [
    'pump',
    'slider',
    'fan',
    'valve'
]

paths = [BASE_FOLDER
         +'dataset/MEL_to_Pandas/data_6dB_{}/FEpandas_MELv1_nm80_ch0.pkl'.format(machine) 
         for machine in machines]

tasks = [{
    'path_descr':path, 
    'feat':{'function':'frame', 'frames':5}, 
    'feat_col':'MELv1_nm80_ch0', 
    'SNR':'6dB', 
    'machine':machine, 
    'ID':ID,
    'BASE_FOLDER':BASE_FOLDER
    } for ID in IDs for machine, path in zip(machines, paths)]

pipes = [Pipe(preprocessing, modeling) for i in range(len(tasks))]

In [5]:
#lw = LoggerWrap()

# create the threads
n_jobs = 4
worker_list = []
queue = Queue()
for worker in range(n_jobs):
    worker = PipeThread(queue)
    worker.daemon = True
    worker.start()
    worker_list.append(worker)

In [6]:
# fill the Queue
#lw.log('multithread mode filling the queue' )
for pipe, task in (zip(pipes, tasks)):
    queue.put((pipe, task))

...loading data
...loading data
...loading data
...loading data


In [9]:
for worker in worker_list:
    worker.stop = True

 93%|██████████████████████████████████████████████████████████████████████████      | 659/712 [00:32<00:02, 24.42it/s]

data loading completed

...preprocessing data


100%|████████████████████████████████████████████████████████████████████████████████| 712/712 [00:33<00:00, 21.49it/s]


data loading completed

...preprocessing data


In [8]:
#lw.log('multithread mode all threads done' ) 
joinlist = outport_akkulist_join(exdia_list=edl) 
outport_akkulist_tofile(base_folder, target_folder, joinlist, m, snr, id)
#lw.log('multithread mode list joined and pickled for the id' )

NameError: name 'edl' is not defined

In [ ]:
# Example settings
n_samples = 200
outliers_fraction = 0.25
clusters_separation = [0, 1, 2]

# define two outlier detection tools to be compared
classifiers = {
    "One-Class SVM": svm.OneClassSVM(nu=0.95 * outliers_fraction + 0.05,
                                     kernel="rbf", gamma=0.1),
    "Robust covariance": EllipticEnvelope(contamination=outliers_fraction),
    "Isolation Forest": IsolationForest(max_samples=n_samples,
                                        contamination=outliers_fraction,
                                        random_state=rng)}

for i, (clf_name, clf) in enumerate(classifiers.items()):
    # fit the data and tag outliers
    clf.fit(X)
    scores_pred = clf.decision_function(X)
    threshold = stats.scoreatpercentile(scores_pred,
                                        100 * outliers_fraction)
    y_pred = clf.predict(X)
    n_errors = (y_pred != ground_truth).sum()
    # plot the levels lines and the points
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    subplot = plt.subplot(1, 3, i + 1)
    subplot.contourf(xx, yy, Z, levels=np.linspace(Z.min(), threshold, 7),
                        cmap=plt.cm.Blues_r)
    a = subplot.contour(xx, yy, Z, levels=[threshold],
                        linewidths=2, colors='red')
    subplot.contourf(xx, yy, Z, levels=[threshold, Z.max()],
                        colors='orange')
    b = subplot.scatter(X[:-n_outliers, 0], X[:-n_outliers, 1], c='white')
    c = subplot.scatter(X[-n_outliers:, 0], X[-n_outliers:, 1], c='black')
    subplot.axis('tight')
    subplot.legend(
        [a.collections[0], b, c],
        ['learned decision function', 'true inliers', 'true outliers'],
        prop=matplotlib.font_manager.FontProperties(size=11),
        loc='lower right')
    subplot.set_title("%d. %s (errors: %d)" % (i + 1, clf_name, n_errors))
    subplot.set_xlim((-7, 7))
    subplot.set_ylim((-7, 7))
plt.subplots_adjust(0.04, 0.1, 0.96, 0.92, 0.1, 0.26)

plt.show()

  0%|                                                                                          | 0/712 [00:00<?, ?it/s]

  0%|                                                                                          | 0/872 [00:00<?, ?it/s]


  0%|                                                                                  | 1/712 [00:00<01:51,  6.37it/s]

  0%|                                                                                  | 1/872 [00:00<02:13,  6.54it/s]


  6%|████▊                                                                            | 51/863 [00:00<01:34,  8.56it/s]

  1%|▊                                                                                 | 9/872 [00:00<01:35,  9.01it/s]


  3%|██                                                                               | 18/712 [00:00<00:57, 11.99it/s]

  2%|█▋                                                                               | 18/872 [00:00<01:09, 12.24it/s]


 18%|██████████████▏        

 20%|████████████████▏                                                               | 144/712 [00:02<00:08, 70.56it/s]

 25%|████████████████████▎                                                           | 153/604 [00:02<00:06, 71.71it/s]


 21%|█████████████████▏                                                              | 153/712 [00:02<00:07, 73.60it/s]

 27%|█████████████████████▎                                                          | 161/604 [00:02<00:06, 72.01it/s]


 18%|██████████████▊                                                                 | 161/872 [00:02<00:09, 71.97it/s]

 28%|██████████████████████▍                                                         | 169/604 [00:02<00:06, 70.69it/s]


100%|███████████████████████████████████████████████████████████████████████████████| 286/286 [00:00<00:00, 497.39it/s]


 20%|████████████████▏                                                               | 177/872 [00:02<00:10, 66.31it/s]

 21%|████████████████▉      

data loading completed

...preprocessing data




 22%|█████████████████▋                                                              | 193/872 [00:02<00:09, 69.80it/s]

 23%|██████████████████▍                                                             | 201/872 [00:02<00:09, 70.46it/s]

 24%|███████████████████▏                                                            | 209/872 [00:03<00:10, 65.37it/s]

 25%|████████████████████                                                            | 219/872 [00:03<00:09, 71.85it/s]

 26%|████████████████████▉                                                           | 228/872 [00:03<00:08, 74.76it/s]

 27%|█████████████████████▋                                                          | 236/872 [00:03<00:08, 72.92it/s]

 28%|██████████████████████▍                                                         | 244/872 [00:03<00:08, 71.69it/s]

 29%|███████████████████████▏                                                        | 253/872 [00:03<00:08, 75.20it/s]

 30%|███████████████████████▉ 

 62%|██████████████████████████████████████████████████                              | 445/712 [00:27<00:05, 48.00it/s]

 64%|██████████████████████████████████████████████████▉                             | 453/712 [00:27<00:05, 46.71it/s]

 65%|███████████████████████████████████████████████████▊                            | 461/712 [00:28<00:04, 53.27it/s]

 66%|████████████████████████████████████████████████████▋                           | 469/712 [00:28<00:04, 59.07it/s]

 67%|█████████████████████████████████████████████████████▋                          | 478/712 [00:28<00:03, 65.86it/s]

 69%|██████████████████████████████████████████████████████▊                         | 488/712 [00:28<00:03, 72.42it/s]

 70%|███████████████████████████████████████████████████████▊                        | 497/712 [00:28<00:02, 71.95it/s]

 71%|████████████████████████████████████████████████████████▋                       | 505/712 [00:28<00:02, 72.77it/s]

 72%|███████████████████████████

 14%|███████████▎                                                                    | 115/814 [00:01<00:10, 66.25it/s]

 15%|████████████                                                                    | 123/814 [00:01<00:09, 69.49it/s]

 16%|████████████▊                                                                   | 131/814 [00:01<00:09, 70.43it/s]

 17%|█████████████▊                                                                  | 140/814 [00:01<00:09, 72.96it/s]

 18%|██████████████▌                                                                 | 148/814 [00:02<00:10, 61.90it/s]

 19%|███████████████▍                                                                | 157/814 [00:02<00:09, 67.68it/s]

 20%|████████████████▎                                                               | 166/814 [00:02<00:08, 72.24it/s]

 22%|█████████████████▎                                                              | 176/814 [00:02<00:08, 76.98it/s]

 23%|██████████████████▏        

 47%|█████████████████████████████████████▉                                          | 386/814 [00:05<00:06, 62.50it/s]

 49%|██████████████████████████████████████▉                                         | 396/814 [00:05<00:06, 69.26it/s]

 50%|███████████████████████████████████████▋                                        | 404/814 [00:05<00:05, 70.63it/s]

 51%|████████████████████████████████████████▌                                       | 413/814 [00:05<00:05, 73.65it/s]

 52%|█████████████████████████████████████████▍                                      | 421/814 [00:05<00:05, 71.41it/s]

 53%|██████████████████████████████████████████▏                                     | 429/814 [00:05<00:05, 72.38it/s]

 54%|██████████████████████████████████████████▉                                     | 437/814 [00:06<00:05, 70.20it/s]

 46%|████████████████████████████████████▊                                           | 328/712 [00:04<00:06, 57.38it/s]

 47%|███████████████████████████

data loading completed

...preprocessing data



 91%|████████████████████████████████████████████████████████████████████████▉       | 742/814 [00:10<00:01, 71.10it/s]
